# TEXTO

Este cuaderno sirve como material de apoyo para los temas tratados en el **Capítulo 22: Procesamiento del lenguaje natural** del libro *Inteligencia artificial: un enfoque moderno*. Este cuaderno utiliza implementaciones de [text.py](https://github.com/aimacode/aima-python/blob/master/text.py).

In [1]:
from text import *
from utils import open_data
from notebook import psource

## CONTENIDO

* Modelos de texto
* Segmentación de texto de Viterbi
* Recuperación de información
* Extracción de información
* Decodificadores

## MODELOS DE TEXTO

Antes de comenzar a analizar algoritmos de procesamiento de texto, necesitaremos crear algunos modelos de lenguaje. Esos modelos sirven como una tabla de búsqueda de probabilidades de caracteres o palabras (según el tipo de modelo). Estos modelos pueden darnos las probabilidades de que aparezcan palabras o secuencias de caracteres en el texto. Tomemos como ejemplo "el". Los modelos de texto pueden darnos la probabilidad de "el", *P("el")*, ya sea como una palabra o como una secuencia de caracteres ("t" seguida de "h" seguida de "e"). La primera representación se llama "modelo de palabras" y trata de palabras como objetos distintos, mientras que la segunda es un "modelo de caracteres" y trata de secuencias de caracteres como objetos. Tenga en cuenta que podemos especificar el número de palabras o la longitud de las secuencias de caracteres para satisfacer mejor nuestras necesidades. Entonces, dado que el número de palabras es igual a 2, tenemos probabilidades en la forma *P(palabra1, palabra2)*. Por ejemplo, *P("de", "el")*. Para los modelos char, hacemos lo mismo pero para los chars.

También es útil almacenar las probabilidades condicionales de palabras dadas las palabras anteriores. Eso significa que, dado que encontramos las palabras "de" y "the", ¿cuál es la probabilidad de que la siguiente palabra sea "mundo"? Más formalmente, *P("mundo"|"de", "el")*. Generalizando, *P(Wi|Wi-1, Wi-2, ... , Wi-n)*.

Llamamos al modelo de palabras *Modelo de palabras N-Gram* (del griego "gram", la raíz de "escribir" o la palabra para "letra") y al modelo de caracteres *Modelo de caracteres N-Gram*. En el caso especial donde *N* es 1, llamamos a los modelos *Modelo de palabras de Unigram* y *Modelo de caracteres de Unigram* respectivamente.

En el módulo `text` implementamos los dos modelos (tanto sus variantes de unigrama como de n-grama) heredando de `CountingProbDist` de `learning.py`. Tenga en cuenta que "CountingProbDist" no devuelve la probabilidad real de cada objeto, sino el número de veces que aparece en nuestros datos de prueba.

Para modelos de palabras tenemos `UnigramWordModel` y `NgramWordModel`. Les proporcionamos un archivo de texto y les muestran la frecuencia de las diferentes palabras. Tenemos `UnigramCharModel` y `NgramCharModel` para los modelos de personajes.

Ejecute las celdas siguientes para ver el código.

In [ ]:
psource(UnigramWordModel, NgramWordModel, UnigramCharModel, NgramCharModel)

A continuación construimos nuestros modelos. El archivo de texto que usaremos para construirlos es *Flatland*, de Edwin A. Abbott. Lo cargaremos desde [here](https://github.com/aimacode/aima-data/blob/a21fc108f52ad551344e947b0eb97df82f8d2b2b/EN-text/flatland.txt). En ese directorio puede encontrar otros archivos de texto que podríamos usar aquí.

### Obteniendo probabilidades

Aquí veremos cómo leer texto y encontrar las probabilidades de cada modelo, y cómo recuperarlas.

Primero la palabra modelos:

In [2]:
flatland = open_data("EN-text/flatland.txt").read()
wordseq = words(flatland)

P1 = UnigramWordModel(wordseq)
P2 = NgramWordModel(2, wordseq)

print(P1.top(5))
print(P2.top(5))

print(P1['an'])
print(P2[('i', 'was')])

[(2081, 'the'), (1479, 'of'), (1021, 'and'), (1008, 'to'), (850, 'a')]
[(368, ('of', 'the')), (152, ('to', 'the')), (152, ('in', 'the')), (86, ('of', 'a')), (80, ('it', 'is'))]
0.0036724740723330495
0.00114584557527324


Vemos que la palabra más utilizada en *Planilandia* es 'the', con 2081 apariciones, mientras que la secuencia más utilizada es 'of the' con 368 apariciones. Además, la probabilidad de 'un' es aproximadamente 0,003, mientras que para 'yo era' es cercana a 0,001. Tenga en cuenta que las cadenas utilizadas como claves están todas en minúsculas. Para el modelo de unigrama, las claves son cadenas simples, mientras que para los modelos de n-gramas tenemos n-tuplas de cadenas.

A continuación, veremos cómo podemos obtener información de las probabilidades condicionales del modelo y cómo podemos generar la siguiente palabra en una secuencia.

In [3]:
flatland = open_data("EN-text/flatland.txt").read()
wordseq = words(flatland)

P3 = NgramWordModel(3, wordseq)

print("Conditional Probabilities Table:", P3.cond_prob[('i', 'was')].dictionary, '\n')
print("Conditional Probability of 'once' give 'i was':", P3.cond_prob[('i', 'was')]['once'], '\n')
print("Next word after 'i was':", P3.cond_prob[('i', 'was')].sample())

Conditional Probabilities Table: {'now': 2, 'glad': 1, 'keenly': 1, 'considered': 1, 'once': 2, 'not': 4, 'in': 2, 'by': 1, 'simulating': 1, 'intoxicated': 1, 'wearied': 1, 'quite': 1, 'certain': 2, 'sitting': 1, 'to': 2, 'rapidly': 1, 'will': 1, 'describing': 1, 'allowed': 1, 'at': 2, 'afraid': 1, 'covered': 1, 'approaching': 1, 'standing': 1, 'myself': 1, 'surprised': 1, 'unusually': 1, 'rapt': 1, 'pleased': 1, 'crushed': 1} 

Conditional Probability of 'once' give 'i was': 0.05128205128205128 

Next word after 'i was': wearied


Primero imprimimos todas las palabras posibles que vienen después de 'yo era' y las veces que han aparecido en el modelo. A continuación, imprimimos la probabilidad de que aparezca 'una vez' después de 'yo era' y, finalmente, elegimos una palabra para continuar después de 'yo era'. Tenga en cuenta que la palabra se elige según su probabilidad de aparecer (un recuento alto de apariciones significa una mayor probabilidad de ser elegida).

Echemos un vistazo a los dos modelos de personajes:

In [4]:
flatland = open_data("EN-text/flatland.txt").read()
wordseq = words(flatland)

P1 = UnigramCharModel(wordseq)
P2 = NgramCharModel(2, wordseq)

print(P1.top(5))
print(P2.top(5))

print(P1['z'])
print(P2[('g', 'h')])

[(19208, 'e'), (13965, 't'), (12069, 'o'), (11702, 'a'), (11440, 'i')]
[(5364, (' ', 't')), (4573, ('t', 'h')), (4063, (' ', 'a')), (3654, ('h', 'e')), (2967, (' ', 'i'))]
0.0006028715031814578
0.0032371578540395666


La letra más común es 'e', ​​aparece más de 19000 veces y la secuencia más común es "\_t". Es decir, un espacio seguido de una 't'. Tenga en cuenta que aunque no contamos los espacios para los modelos de palabras o los modelos de caracteres unigramas, sí los contamos para los modelos de caracteres de n-gramas.

Además, la probabilidad de que aparezca la letra 'z' es cercana a 0,0006, mientras que para el bigrama 'gh' es de 0,003.

### Generando muestras

Además de leer las probabilidades de n-gramas, también podemos usar nuestro modelo para generar secuencias de palabras, usando la función "muestras" en los modelos de palabras.

In [5]:
flatland = open_data("EN-text/flatland.txt").read()
wordseq = words(flatland)

P1 = UnigramWordModel(wordseq)
P2 = NgramWordModel(2, wordseq)
P3 = NgramWordModel(3, wordseq)

print(P1.samples(10))
print(P2.samples(10))
print(P3.samples(10))

hearing as inside is confined to conduct by the duties
all and of voice being in a day of the
party they are stirred to mutual warfare and perish by


En el modelo de unigrama, la mayoría de las veces obtenemos galimatías, ya que cada palabra se selecciona según su frecuencia de aparición en el texto, sin tener en cuenta las palabras anteriores. Sin embargo, a medida que aumentamos *n*, comenzamos a obtener muestras que tienen cierta apariencia de coherencia y recuerdan un poco al inglés normal. A medida que aumentemos nuestros datos, estas muestras mejorarán.

Vamos a intentarlo. Agregaremos al modelo más datos con los que trabajar y veremos qué sale.

In [6]:
data = open_data("EN-text/flatland.txt").read()
data += open_data("EN-text/sense.txt").read()

wordseq = words(data)

P3 = NgramWordModel(3, wordseq)
P4 = NgramWordModel(4, wordseq)
P5 = NgramWordModel(5, wordseq)
P7 = NgramWordModel(7, wordseq)

print(P3.samples(15))
print(P4.samples(15))
print(P5.samples(15))
print(P7.samples(15))

leave them at cleveland this christmas now pray do not ask you to relate or
meaning and both of us sprang forward in the direction and no sooner had they
palmer though very unwilling to go as well from real humanity and good nature as
time about what they should do and they agreed he should take orders directly and


Observe cómo las muestras comienzan a volverse cada vez más razonables a medida que agregamos más datos y aumentamos el parámetro *n*. Todavía estamos muy lejos de la generación de texto realista, pero al mismo tiempo podemos ver que con suficientes datos incluso los algoritmos rudimentarios pueden producir algo casi aceptable.

## SEGMENTACIÓN DE TEXTO DE VITERBI

### Descripción general

Nos dan una cadena que contiene palabras de una oración, ¡pero todos los espacios han desaparecido! Es muy difícil de leer y nos gustaría separar las palabras de la cadena. Podemos lograr esto empleando el algoritmo de "Segmentación de Viterbi". Toma como entrada la cadena a segmentar y un modelo de texto, y devuelve una lista de palabras separadas.

El algoritmo opera en un enfoque de programación dinámica. Comienza desde el principio de la cadena y construye iterativamente la mejor solución utilizando soluciones anteriores. Lo logra segmentando la cadena en "ventanas", cada ventana representa una palabra (real o galimatías). Luego calcula la probabilidad de que ocurra la secuencia hasta esa ventana/palabra y actualiza su solución. Cuando termina, rastrea desde la última palabra y encuentra la secuencia completa de palabras.

### Implementación

In [ ]:
psource(viterbi_segment)

La función toma como entrada una cadena y un modelo de texto, y devuelve la secuencia de palabras más probable, junto con la probabilidad de esa secuencia.

La "ventana" es `w` e incluye los caracteres desde *j* hasta *i*. Lo usamos para "construir" la siguiente secuencia: desde el principio hasta *j* y luego `w`. Anteriormente hemos calculado la probabilidad desde el inicio hasta *j*, así que ahora multiplicamos esa probabilidad por `P[w]` to get the probability of the whole sequence. If that probability is greater than the probability we have calculated so far for the sequence from the start to *i* (`best[i]`), la actualizamos.

### Ejemplo

El modelo que utiliza el algoritmo es `UnigramTextModel`. Primero construiremos el modelo usando el texto *Flatland* y luego intentaremos separar una oración sin espacios.

In [3]:
flatland = open_data("EN-text/flatland.txt").read()
wordseq = words(flatland)
P = UnigramWordModel(wordseq)
text = "itiseasytoreadwordswithoutspaces"

s, p = viterbi_segment(text,P)
print("Sequence of words is:",s)
print("Probability of sequence is:",p)

Sequence of words is: ['it', 'is', 'easy', 'to', 'read', 'words', 'without', 'spaces']
Probability of sequence is: 2.273672843573388e-24


El algoritmo recuperó correctamente las palabras de la cadena. También nos dio la probabilidad de esta secuencia, que es pequeña, pero sigue siendo la segmentación más probable de la cuerda.

## RECUPERACIÓN DE INFORMACIÓN

### Descripción general

Con **Recuperación de información (IR)** encontramos documentos que son relevantes para las necesidades de información del usuario. Un ejemplo popular es un motor de búsqueda web, que encuentra y presenta al usuario páginas relevantes para una consulta. Sin embargo, la recuperación de información no se limita sólo a la devolución de documentos, sino que también puede utilizarse para otro tipo de consultas. Por ejemplo, responder preguntas cuando la consulta es una pregunta, devolver información cuando la consulta es un concepto y muchas otras aplicaciones. Un sistema de infrarrojos se compone de lo siguiente:

* Un cuerpo (llamado corpus) de documentos: Una colección de documentos, donde trabajará el RI.

* Un lenguaje de consulta: Una consulta representa lo que el usuario quiere.

* Resultados: Los documentos que el sistema califica como relevantes para la consulta y las necesidades del usuario.

* Presentación de los resultados: Cómo se presentan los resultados al usuario.

¿Cómo determina un sistema de IR qué documentos son relevantes? Podemos firmar un documento como relevante si en él aparecen todas las palabras de la consulta, y firmarlo como irrelevante en caso contrario. Incluso podemos ampliar el lenguaje de consulta para que admita operaciones booleanas (por ejemplo, "pintar Y pincel") y luego firmar como relevante el resultado de la consulta para el documento. Sin embargo, esta técnica no proporciona un nivel de relevancia. Todos los documentos son relevantes o irrelevantes, pero en realidad algunos documentos son más relevantes que otros.

Entonces, en lugar de un sistema de relevancia booleano, usamos una *función de puntuación*. Existen muchas funciones de puntuación para muchas situaciones diferentes. Uno de los más utilizados tiene en cuenta la frecuencia de las palabras que aparecen en un documento, la frecuencia con la que aparece una palabra en todos los documentos (por ejemplo, la palabra "a" aparece mucho, por lo que no es muy importante) y la longitud de un documento (ya que los documentos grandes tendrán más apariciones de los términos de consulta, pero un documento corto con muchas apariciones parece muy relevante). Combinamos estas propiedades en una fórmula y obtenemos una puntuación numérica para cada documento, de modo que luego podamos cuantificar la relevancia y elegir los mejores documentos.

Sin embargo, estas funciones de puntuación no son perfectas y se pueden mejorar. Por ejemplo, para la función de puntuación anterior asumimos que cada palabra es independiente. Sin embargo, ese no es el caso, ya que las palabras pueden compartir significado. Por ejemplo, las palabras "pintor" y "pintores" están estrechamente relacionadas. Si en una consulta tenemos la palabra "pintor" y en un documento aparece mucho la palabra "pintores", esto puede ser un indicativo de que el documento es relevante pero nos lo estamos perdiendo ya que solo buscamos "pintor". Hay muchas maneras de combatir esto. Uno de ellos es reducir las palabras de consulta y documento en sus raíces. Por ejemplo, tanto "pintor" como "pintores" tienen "pintura" como forma de raíz. Esto puede mejorar ligeramente el rendimiento de los algoritmos.

Para determinar qué tan bueno es un sistema IR, le damos al sistema un conjunto de consultas (para las cuales conocemos las páginas relevantes de antemano) y registramos los resultados. Las dos medidas de rendimiento son *precisión* y *recuperación*. La precisión mide la proporción de documentos de resultados que realmente son relevantes. La recuperación mide la proporción de documentos relevantes (que, como se mencionó anteriormente, conocemos de antemano) que aparecen en los documentos de resultados.

### Implementación

Puede leer el código fuente ejecutando el siguiente comando:

In [2]:
psource(IRSystem)

El argumento "palabras vacías" indica palabras en las consultas que no deben contabilizarse en los documentos. Suelen ser palabras muy comunes que no aportan ninguna información significativa sobre la relevancia de un documento.

Una guía rápida para las funciones de la clase `IRSystem`:

* `index_document`: Agrega un documento a la colección de documentos (llamada `documents`), que es una lista de tuplas. Además, cuente cuántas veces aparece cada palabra de la consulta en cada documento.

* `index_collection`: indexa una colección de documentos dada por `filenames`.

* `query`: Devuelve una lista de `n` pares de `(score, docid)` ordenados según la puntuación de cada documento. También se encarga de la consulta especial "aprender: X", donde en lugar de la funcionalidad normal presentamos la salida del comando de terminal "X".

* `score`: Califica un documento dado para la palabra dada usando `log(1+k)/log(1+n)`, donde `k` es el número de palabras de consulta en un documento y `k` es el total número de palabras del documento. Se pueden utilizar otras funciones de puntuación y usted puede sobrescribir esta función para adaptarla mejor a sus necesidades.

* `total_score`: Calcula la suma de todas las palabras de consulta en un documento determinado.

* `present`/`present_results`: Presenta los resultados como una lista.

También tenemos la clase "Documento" que contiene metadatos de documentos, como su título, URL y número de palabras. Se puede utilizar una clase adicional, `UnixConsultant`, para inicializar un sistema IR para manuales de comandos de Unix. Este es el ejemplo que usaremos para mostrar la implementación.

### Ejemplo

Primero, echemos un vistazo al código fuente de "UnixConsultant".

In [3]:
psource(UnixConsultant)

La clase crea un sistema IR con las palabras vacías "¿Cómo hago la a de?". Podríamos agregar más palabras para excluir, pero las consultas que probaremos generalmente tendrán ese formato, por lo que es conveniente. Después de la inicialización del sistema, obtenemos los archivos manuales y comenzamos a indexarlos.

Construyamos nuestro consultor de Unix y ejecutemos una consulta:

In [4]:
uc = UnixConsultant()

q = uc.query("how do I remove a file")

top_score, top_doc = q[0][0], q[0][1]
print(top_score, uc.documents[top_doc].url)

0.7682667868462166 aima-data/MAN/rm.txt


Preguntamos cómo eliminar un archivo y el principal resultado fue el manual `rm` (el comando de Unix para eliminar). ¡Esto es exactamente lo que queríamos! Probemos con otra consulta:

In [5]:
q = uc.query("how do I delete a file")

top_score, top_doc = q[0][0], q[0][1]
print(top_score, uc.documents[top_doc].url)

0.7546722691607105 aima-data/MAN/diff.txt


Aunque básicamente pedimos lo mismo, obtuvimos un resultado superior diferente. El comando `diff` muestra las diferencias entre dos archivos. Entonces el sistema nos falló y nos presentó un documento irrelevante. ¿Porqué es eso? Desafortunadamente nuestro sistema IR considera cada palabra independiente. "Eliminar" y "eliminar" tienen significados similares, pero como son palabras diferentes, nuestro sistema no establecerá la conexión. Entonces, el manual `diff` que menciona mucho la palabra `delete` recibe el visto bueno por delante de otros manuales, mientras que el manual `rm` no está en el conjunto de resultados ya que no usa la palabra en absoluto.

## EXTRACCIÓN DE INFORMACIÓN

**Extracción de información (IE)** es un método para encontrar apariciones de clases de objetos y relaciones en el texto. A diferencia de los sistemas IR, un sistema IE incluye nociones (limitadas) de sintaxis y semántica. Si bien es difícil extraer información de objetos en un entorno general, para dominios más específicos el sistema es muy útil. Un modelo de sistema IE utiliza plantillas que coinciden con cadenas en un texto.

Un ejemplo típico de este modelo es la lectura de precios en páginas web. Los precios suelen aparecer después de un dólar y constan de números, tal vez seguidos de dos puntos decimales. Antes del precio, normalmente aparecerá una cadena como "precio:". Construyamos una plantilla de muestra.

Con la siguiente expresión regular (*regex*) podemos extraer precios del texto:

`[$][0-9]+([.][0-9][0-9])?`

Donde `+` significa 1 o más apariciones y `?` significa como máximo 1 aparición. Por lo general, una plantilla consta de un prefijo, un destino y una expresión regular de postfijo. En esta plantilla, la expresión regular del prefijo puede ser "precio:", la expresión regular de destino puede ser la expresión regular anterior y la expresión regular del postfijo puede estar vacía.

Una plantilla puede coincidir con varias cadenas. Si este es el caso, necesitamos una forma de resolver las coincidencias múltiples. En lugar de tener una sola plantilla, podemos usar varias plantillas (ordenadas por prioridad) y elegir la que coincida con la plantilla de mayor prioridad. También podemos utilizar otras formas de elegir. Para el ejemplo del dólar, podemos elegir la coincidencia más cercana a la mitad numérica de la coincidencia más alta. Para el texto "Precio $90, oferta especial $70, envío $5", elegiríamos "$70", ya que está más cerca de la mitad de la igualación más alta ("$90").

Lo anterior se llama extracción *basada en atributos*, donde queremos encontrar atributos en el texto (en el ejemplo, el precio). Un sistema de extracción más sofisticado tiene como objetivo tratar con múltiples objetos y las relaciones entre ellos. Cuando dicho sistema lee el texto "$100", debería determinar no sólo el precio sino también qué objeto tiene ese precio.

Los sistemas de extracción de relaciones se pueden construir como una serie de autómatas de estados finitos. Cada autómata recibe como entrada texto, realiza transformaciones en el texto y lo pasa al siguiente autómata como entrada. La configuración de un autómata puede constar de las siguientes etapas:

1. **Tokenización**: Segmenta el texto en tokens (palabras, números y puntuación).

2. **Manejo de palabras complejas**: Maneja palabras complejas como "ríndete" o incluso nombres como "Smile Inc."

3. **Manejo de grupos básicos**: Maneja grupos de sustantivos y verbos, segmentando el texto en cadenas de verbos o sustantivos (por ejemplo, "tuve que rendirme").

4. **Manejo de frases complejas**: Maneja frases complejas usando reglas gramaticales de estados finitos. Por ejemplo, "Humano+Jugó Ajedrez("con" Humano+)?" puede ser una plantilla/regla para capturar la relación de alguien jugando al ajedrez con otros.

5. **Fusión de estructuras**: Fusiona las estructuras construidas en los pasos anteriores.

Los modelos de extracción de información de estado finito basados ​​en plantillas funcionan bien para dominios restringidos, pero funcionan mal a medida que el dominio se vuelve cada vez más general. Sin embargo, hay muchos modelos para elegir, cada uno con sus propias fortalezas y debilidades. Algunos de los modelos son los siguientes:

* **Probabilístico**: Usando modelos ocultos de Markov, podemos extraer información en forma de prefijo, objetivo y sufijo de un texto determinado. Dos ventajas de usar HMM sobre plantillas es que podemos entrenar HMM a partir de datos y no necesitamos diseñar plantillas elaboradas, y que un enfoque probabilístico se comporta bien incluso con ruido. En una expresión regular, si un carácter está fuera de lugar, no tenemos coincidencia, mientras que con un enfoque probabilístico tenemos un proceso más fluido.

* **Campos aleatorios condicionales**: Un problema con los HMM es la suposición de independencia estatal. Los CRF son muy similares a los HMM, pero no tienen la restricción de estos últimos. Además, los CRF utilizan *funciones de características*, que actúan como pesos de transición. Por ejemplo, si para la observación $e_{i}$ y el estado $x_{i}$ tenemos que $e_{i}$ es "correr" y $x_{i}$ es el estado ATLETA, podemos tener $f( x_{i}, e_{i}) = 1$ e igual a 0 en caso contrario. Podemos usar múltiples funciones superpuestas e incluso podemos usar funciones para transiciones de estado. Las funciones de características no tienen que ser binarias (como en el ejemplo anterior), pero también pueden tener un valor real. Además, podemos usar cualquier $e$ para la función, no solo la observación actual. Para reunirlo todo, sopesamos una transición según la suma de características.

* **Extracción de Ontología**: Este es un método para recopilar información y hechos en un dominio general. Un hecho puede tener la forma "NP es NP", donde "NP" denota una frase nominal. Por ejemplo, "El conejo es un mamífero".

## DECODIFICADORES

### Introducción

En esta sección intentaremos decodificar texto cifrado utilizando modelos de texto probabilísticos. Un texto cifrado se obtiene cifrando un mensaje de texto. Este cifrado nos permite comunicarnos de forma segura, ya que cualquiera que tenga acceso al texto cifrado pero no sepa decodificarlo no podrá leer el mensaje. Restringiremos nuestro estudio a <b>Cifrados de sustitución monoalfabéticos</b>. Estas son formas primitivas de cifrado en las que cada letra del texto del mensaje (también conocido como texto sin formato) se reemplaza por otra letra del alfabeto.

### Descodificador de cambios

#### El cifrado César

El cifrado César, también conocido como cifrado por desplazamiento, es una forma de cifrado por sustitución monoalfabético en el que cada letra se <i>desplaza</i> en un valor fijo. Un cambio de <b>`n`</b> en este contexto significa que cada letra del texto sin formato se reemplaza con una letra correspondiente a `n` letras del alfabeto. Por ejemplo, el texto sin formato `"ABCDWXYZ"` desplazado por `3` produce `"DEFGZABC"`. Observe cómo "X" se convirtió en "A". Esto se debe a que el alfabeto es cíclico, es decir, la letra después de la última letra del alfabeto, "Z", es la primera letra del alfabeto: "A".

In [6]:
plaintext = "ABCDWXYZ"
ciphertext = shift_encode(plaintext, 3)
print(ciphertext)

DEFGZABC


#### Decodificando un cifrado César

Para decodificar un cifrado César aprovechamos el hecho de que no todas las letras del alfabeto se utilizan por igual. Algunas letras se utilizan más que otras y es más probable que algunos pares de letras aparezcan juntas. A un par de letras consecutivas lo llamamos <b>bigrama</b>.

In [7]:
print(bigrams('this is a sentence'))

['th', 'hi', 'is', 's ', ' i', 'is', 's ', ' a', 'a ', ' s', 'se', 'en', 'nt', 'te', 'en', 'nc', 'ce']


Usamos `CountingProbDist` para obtener la distribución de probabilidad de bigramas. En el alfabeto latino consta de sólo "26" letras. Esto limita el número total de sustituciones posibles a "26". Invertimos la codificación de desplazamiento para una `n` dada y verificamos qué tan probable es usando la distribución de bigram. Probamos todos los valores `26` de `n`, es decir, desde `n = 0` hasta `n = 26` y usamos el valor de `n` que proporciona el texto sin formato más probable.

In [7]:
%psource ShiftDecoder

#### Ejemplo

Codifiquemos un mensaje secreto usando el cifrado César y luego intentemos decodificarlo usando `ShiftDecoder`. Usaremos nuevamente `flatland.txt` para construir el modelo de texto.

In [8]:
plaintext = "This is a secret message"
ciphertext = shift_encode(plaintext, 13)
print('The code is', '"' + ciphertext + '"')

The code is "Guvf vf n frperg zrffntr"


In [9]:
flatland = open_data("EN-text/flatland.txt").read()
decoder = ShiftDecoder(flatland)

decoded_message = decoder.decode(ciphertext)
print('The decoded message is', '"' + decoded_message + '"')

The decoded message is "This is a secret message"


### Decodificador de permutación
Intentemos ahora decodificar mensajes cifrados mediante un cifrado de sustitución monoalfabético general. Las letras del alfabeto se pueden reemplazar por cualquier permutación de letras. Por ejemplo, si el alfabeto constaba de `{A B C}`, entonces se puede reemplazar por `{A C B}`, `{B A C}`, `{B C A}`, `{C A B}`, `{C B A}` o incluso `{A B C}` en sí. Supongamos que elegimos la permutación `{C B A}`, entonces el texto plano `"CAB BA AAC"` se convertiría en `"ACB BC CCA"`. Podemos ver que el cifrado César también es una forma de cifrado por permutación donde la permutación es una permutación cíclica. A diferencia del cifrado César, no es posible intentar todas las permutaciones posibles. El número de permutaciones posibles en el alfabeto latino es `26!`, que es del orden $10^{26}$. Utilizamos algoritmos de búsqueda de gráficos para buscar una permutación "buena".

In [ ]:
psource(PermutationDecoder)

Cada estado/nodo en el gráfico se representa como un mapa letra a letra. Si no hay ninguna asignación para una letra, significa que la letra no ha cambiado en la permutación. Estos mapas se almacenan como diccionarios. Cada diccionario es una permutación "potencial". Usamos la palabra "potencial" porque cada diccionario no necesariamente representa una permutación válida ya que una permutación no puede tener elementos repetidos. Por ejemplo, el diccionario `{'A': 'B', 'C': 'X'}` no es válido porque `'A'` se reemplaza por `'B'`, pero también lo es `'B'` porque el El diccionario no tiene una asignación para "B". Dos diccionarios también pueden representar la misma permutación, p. `{'A': 'C', 'C': 'A'}` y `{'A': 'C', 'B': 'B', 'C': 'A'}` representan lo mismo permutación donde `'A'` y `'C'` se intercambian y todas las demás letras permanecen inalteradas. Para garantizar que obtengamos una permutación válida, un estado objetivo debe asignar todas las letras del alfabeto. También evitamos repeticiones en la permutación al permitir solo aquellas acciones que van a un nuevo estado/nodo en el que la letra recién agregada al diccionario se asigna a una letra no asignada anteriormente. Estas dos reglas juntas aseguran que el diccionario de un estado objetivo representará una permutación válida.
La puntuación de un estado se determina utilizando puntuaciones de palabras, puntuaciones de unigramas y puntuaciones de bigramas. Experimente con diferentes ponderaciones para puntuaciones de palabras, unigramas y bigramas y vea cómo afectan la decodificación.

In [11]:
ciphertexts = ['ahed world', 'ahed woxld']

pd = PermutationDecoder(canonicalize(flatland))
for ctext in ciphertexts:
    print('"{}" decodes to "{}"'.format(ctext, pd.decode(ctext)))

"ahed world" decodes to "shed could"
"ahed woxld" decodes to "shew atiow"


Como se desprende del ejemplo anterior, la decodificación de permutaciones utilizando la mejor primera búsqueda es sensible al texto inicial. Esto se debe a que no sólo el diccionario final, con sustituciones de todas las letras, debe tener una buena puntuación, sino también los diccionarios intermedios. Podría pensar en ello como realizar una búsqueda local encontrando sustituciones para cada letra una por una. Podríamos obtener resultados muy diferentes cambiando incluso una sola letra porque esa letra podría ser un factor decisivo para seleccionar la sustitución en las primeras etapas, lo que se multiplica y afecta las etapas posteriores. Para mejorar la búsqueda, podemos utilizar diferentes definiciones de puntuación en diferentes etapas y optimizar qué letra sustituir primero.